<a href="https://colab.research.google.com/github/Subhutii/GRIT-AI/blob/main/KeyPhrase_io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

filename="韓国から中国まとめ.csv"
df = pd.read_csv(filename)

In [ ]:
df_c = df.loc[:,["Country"]]
# print(df_c)
counry=df_c[~df_c.duplicated()]

cnt = df['Country'].str.split(',')
Country = []

for i in cnt:
    if i not in Country:
        Country.append(i)

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
#国名ファイル用　日本語　→　英語
from googletrans import Translator
 
tr = Translator()
CountryText=[]
for i in range(len(Country)):
      result=[i,Country[i]]
      countryText="".join(Country[i])
      result = tr.translate(countryText, src="ja", dest="en").text
      CountryText.append(result)

print(CountryText)

Driveとマウントする必要がある

**国とTextのみのファイル出力**

In [ ]:
import csv

# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/out/"

for i in range(len(Country)):
  rm_Country=[]
  # CSVファイル名＝国名
  countStr= "".join(Country[i])
  csv_name = countStr+(".csv")

  f = open(csv_path+csv_name, 'w',encoding='utf-8_sig',newline='')
  writer = csv.writer(f)


  #国ごとにText抽出
  dfS=df[df['Country'].isin(Country[i])]
  text = dfS['Text'].str.split(',')
  for i in text:
    # print(i)
    rm_Country=[i]
    writer.writerow(rm_Country)
  f.close()


**出力**

In [ ]:
# 国ごとのtext読み込み
# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/out/"

for i in range(len(Country)):
  rm_result=[]
  # CSVファイル名＝国名
  countStr= "".join(Country[i])
  csv_name = countStr+(".csv")

  with open(csv_path+csv_name, newline='') as f:
    lines = f.readlines()

    print(lines)

In [ ]:
!pip install textacy > /dev/null 2>&1

In [ ]:
!pip install -U pip setuptools wheel > /dev/null 2>&1
!pip install -U spacy > /dev/null 2>&1
!python -m spacy download ja_core_news_sm > /dev/null 2>&1
!pip install neologdn > /dev/null 2>&1
!pip install emoji --upgrade > /dev/null 2>&1

In [ ]:
import pandas as pd
import emoji
import numpy as np
import neologdn
import re
import string
import csv
from typing import List, Tuple
import spacy
from spacy.lang.ja import Japanese, DetailedToken
from textacy import extract
import ginza # 動作モード切替のため

In [ ]:
import emoji
import re

def remove_emoji(text):
   '''
   remove all of emojis from text
   '''
   text=  emoji.demojize(text)
   text= re.sub(r'(:[!_\-\w]+:)', '', text)

   return text

In [ ]:
# 前処理
def preprocess(x):
  x=remove_emoji(x)
  #絵文字の除去
  emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"  # emoticons
            "\U0001F300-\U0001F5FF"  # symbols & pictographs
            "\U0001F680-\U0001F6FF"  # transport & map symbols
            "\U0001F1E0-\U0001F1FF"  # flags (iOS)
            "]+",
            flags=re.UNICODE,
        )

  x = emoji_pattern.sub(r"", x)

  x = neologdn.normalize(x)
  x = x.lower()
  x = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", x)#URLの除去
  x = re.sub(r"[!\?%-/:-@[-`{-~]", r" ", x)# 半角記号の置換
  x = re.sub("[■-♯]", "", x)# 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
  x = re.sub(r"(\d)([,.])(\d+)", "\1\3", x)#桁区切り (+ 小数点) 除去
  # x = re.sub(r"\d+", "0", x)#数字を0に置換
  # x = re.sub(r"\d+","" ,x)#数字除去
  x = re.sub(r'[a-zA-Z0-9]', '', x)#英数字除去
  x = re.sub(r"・", ", ", x)#・→，
  x = re.sub(r"[\(\)「」【】]", "", x)# 括弧削除（まだ消すやつある）
  x = re.sub(r"[\！\？＝＋－\+\*\-\&\(\)\.\,/=\-，．。、\ ￥’”→↓←↑…「」（）『 \・～『』・；：⚫※％]","", x)

  

  return x

In [ ]:
#extract後の可視化
def decompose_keyterms(keyterm_list: List[str]) -> Tuple:
    terms = [el[0] for el in keyterm_list]
    scores = [el[1] for el in keyterm_list]
    return terms, scores

#トークン化
def create_doc(text):
  nlp = spacy.load('ja_ginza')
  mode="C"
  ginza.set_split_mode(nlp, mode) # モード切替
  doc = nlp(text)
  return doc



In [ ]:
#textrank
def textrank_algo(doc):
    include_pos=["NOUN", "PROPN", "ADJ"]#英語の品詞タグ名詞形容詞動詞，
    textrank = extract.keyterms.textrank(doc, normalize="lower",include_pos=include_pos,topn=30)
    terms_textrank, scores_textrank = decompose_keyterms(textrank)

    return terms_textrank, scores_textrank

#PositionRank
# def PositionRank_algo(doc):
#     include_pos=["NOUN", "PROPN", "ADJ"]#英語の品詞タグ名詞形容詞動詞，
#     PositionRan = extract.keyterms.textrank(doc, normalize="lemma",include_pos=None,window_size=10, edge_weighting="count", position_bias=True,topn=20)
#     terms_PositionRank, scores_PositionRank = decompose_keyterms(PositionRan)

#     return terms_PositionRank, scores_PositionRank

#yake
def yake_algo(doc):
    include_pos=["NOUN", "PROPN", "ADJ"]#英語の品詞タグ名詞形容詞動詞，
    yake = extract.keyterms.yake(doc, normalize="lower",include_pos=include_pos,ngrams=(2, 3),topn=30)
    terms_yake, scores_yake = decompose_keyterms(yake)

    return terms_yake, scores_yake

#scake
def scake_algo(doc):
    scake = extract.keyterms.scake(doc, normalize="lemma")
    terms_scake, scores_scake = decompose_keyterms(scake)

    return terms_scake, scores_scake

#sgrank
def sgrank_algo(doc):
    include_pos=["NOUN", "PROPN", "ADJ"]#英語の品詞タグ名詞形容詞動詞，
    sgrank = extract.keyterms.sgrank(doc, normalize="lower",include_pos=include_pos,ngrams=(2, 3),topn=30)
    terms_sgrank, scores_sgrank = decompose_keyterms(sgrank)
    return terms_sgrank, scores_sgrank

In [ ]:
#品詞でフィルタリング
def preprocessPOS(text):
  doc = create_doc(text)

  #英語の品詞タグでストップワード作成
  rmPOS=["PUNCT", "NUM", "SYM","PRON","INTJ","SCONJ","AUX","PART","CCONJ","ADV","ADP","SPACE","VERB","DET"]
  filtere = [rmtoken.text for rmtoken in doc if rmtoken.pos_ in rmPOS]
  # print(filtere)
  #日本の品詞タグでストップワード作成
  stopTag=["感動詞-フィラー","感動詞-一般","補助記号-句点","補助記号-一般","補助記号-読点","記号-一般","記号-文字","名詞-数詞","名詞-普通名詞-助数詞可能","	名詞-普通名詞-副詞可能","代名詞","接尾辞-名詞的-助数詞","接尾辞","接尾辞-名詞的-一般","接尾辞-名詞的-副詞可能","接尾辞-名詞的-サ変可能","接頭辞","助動詞","助詞-係助詞","助詞-格助詞","助詞-終助詞","助詞-副助詞","助詞-準体助詞","副詞","連体詞"]
  filter1 = [token.text for token in doc if token.tag_ in stopTag]
  # print(filter1)
  #ストップワード結合
  filtere_all = filtere+filter1

  filtered_doc = [token.text for token in doc if not token.text in filtere_all]

  Str = " ".join(filtered_doc)# list -> str
  
  return Str

**StopWord**

In [ ]:
def stopwords_text():
  stopword_file = open("Newstopwords.txt", "r")
  stopword = []
  w=stopword_file.readlines()
  # print(w)
  for line in w:
    if line != '\n':
      stopword.append(str(line.strip()))

  return stopword

In [ ]:
def rm_stopword(x):
  sl=stopwords_text()
  result=[]

  for token in x.split( ):
    if not token in sl:
      result.append(token)
  w=set(result)
  strResult=  " ".join(w)# list -> str

  return strResult


**csvまとめて分割 ＆ キーフレーズ抽出**

In [ ]:
import csv
# 呼び出し・結果

# CSVファイルの保管先
csv_path1 = "/content/drive/MyDrive/out/TextRank/"
csv_path2 = "/content/drive/MyDrive/out/sgRank/"
csv_path3 = "/content/drive/MyDrive/out/YAKE/"

# CSVファイルの保管先
op_csv='/content/drive/MyDrive/out/'

for i in range(len(Country)):
  # CSVファイル名＝国名
  c_name="".join(CountryText[i])
  countStr= "".join(Country[i])

  with open(op_csv+countStr+(".csv"), newline='') as f:
    lines = f.readlines()

  chunks = [' '.join(lines[i:i+95]) for i in range(0, len(lines), 95)]

  fTextRank = open(csv_path1+c_name+(".csv"), 'w',encoding='utf-8_sig',newline='')
  writer_textRank = csv.writer(fTextRank)

  fYAKE = open(csv_path2+c_name+(".csv"), 'w',encoding='utf-8_sig',newline='')
  writer_yake = csv.writer(fYAKE)

  fsgRank = open(csv_path3+c_name+(".csv"), 'w',encoding='utf-8_sig',newline='')
  writer_sgRank = csv.writer(fsgRank)

  textRank=[]
  yake=[]
  sgRank=[]
  PositionRank=[]
  
  for chunk in chunks:
    # rmPOS=preprocessPOS(chunk)
    # pre = preprocess(rmPOS)
    # doc = create_doc(pre)

    rmPOS="".join(chunk)
    pre = preprocess(rmPOS)
    doc = create_doc(pre)

    terms_textrank, score_textrank = textrank_algo(doc)
    terms_yake, score_yake = yake_algo(doc)
    # terms_scake, score_scake = scake_algo(doc)
    terms_sgrank, scores_sgrank = sgrank_algo(doc)
    # terms_PositionRank, scores_PositionRank = PositionRank_algo(doc)

    textRank+=terms_textrank
    yake+=terms_yake
    sgRank+=terms_sgrank
    # PositionRank+=terms_PositionRank

    setResult1=set(textRank)
    result1=[]
    for a in setResult1:
      rm=rm_stopword(a)
      result1+=[rm]

    setResult2=set(yake)
    result2=[]
    for j in setResult2:
      rm2=rm_stopword(j)
      result2+=[rm2]

    setResult3=set(sgRank)
    result3=[]
    for k in setResult3:
      rm3=rm_stopword(k)
      result3+=[rm3]
      
    result1=list(filter(None, set(result1)))
    result2=list(filter(None, set(result2)))
    result3=list(filter(None, set(result3)))

    for t in result1:
      t1=[countStr,t]
      writer_textRank.writerow(t1)

    for y in result2:
      y1=[countStr,y]
      writer_yake.writerow(y1)

    for s in result3:
      s1=[countStr,s]
      writer_sgRank.writerow(s1)

  fTextRank.close()
  fYAKE.close()
  fsgRank.close()


  print("textrank "+countStr, result1)
  print("yake "+countStr,result2)
  print("sgrank "+countStr,result3)
# print("PositionRank ",PositionRank)

**spacy**

In [ ]:
#spacy
op_csv='/content/drive/MyDrive/out/'
countStr="オーストラリア"
def run(line):
  with open(op_csv+countStr+(".csv"), newline='') as f:
    lines = f.readlines()
  chunks = [' '.join(line[i:i+100]) for i in range(0, len(line), 100)]
  taken=[]
  sl=stopwords_text()
  for tok in chunks:
    # rmPOS=preprocessPOS(tok)
    # text = preprocess(rmPOS)
    # doc = create_doc(text)

    rmPOS="".join(tok)
    # text = preprocess(rmPOS)
    doc = create_doc(rmPOS)

    for tok in doc.ents:
      if tok.text:
        print(tok.text,"   ",tok.label_)
        taken.append(tok.text)

  print(set(taken))

  # sl=stopwords_text()
  # result=[]

  # for token in x.split( ):
  #   # print(token)
  #   if not token in sl:
  #     result.append(token)
  # w=set(result)
  # strResult=  " ".join(w)# list -> str 
  # # print(strResult)

In [ ]:
run(lines)

In [ ]:
spacy.explain("GPE")#xpacyのラベル確認